In [63]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import os
import state_code
from state_code import SAMPLE
import collections
import plotly.graph_objects as go
import numpy as np

In [64]:
source = requests.get('https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports/')

soup = bs(source.text,'html.parser')
page = soup.prettify()

In [66]:
web_fileset = set()
web_set = set()
dic_p = {}
for i in soup.find_all('a', class_='js-navigation-open'):
    if 'csv' in i['href']:
        web_set.add(i['href'])
        web_fileset.add(i['href'].split('/')[-1])
        dic_p[i['href'].split('/')[-1]] = i['href']

dir_set = set(os.listdir('./dataset'))
new_data = web_fileset.difference(dir_set)
for i in new_data:
    path = 'https://raw.githubusercontent.com'+''.join(dic_p[i].split('/blob'))
    # raw_source = requests.get(path)
    # raw_soup = bs(raw_source.text, 'html.parser')
    # csv_path = 'https://raw.githubusercontent.com'+raw_soup.find(id='raw-url')['href']
    print(path)
    df = pd.read_csv(path)
    filename = i.split('/')[-1]
    #print(filename)
    df.to_csv('./dataset/'+filename)


https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/02-28-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-20-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-22-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-20-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-11-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports

In [55]:
files = os.listdir('./dataset/')
files.sort()

In [61]:
biggest_con = 0

def set_color_group(item):
    #print(lst)
    pop = int(item)
    [1000, 5000, 10000, 50000]
    if pop <= 0:
        return 0
    elif pop < 1000:
        return 1
    elif 1000 <= pop < 5000:
        return 2
    elif 5000 <= pop < 10000:
        return 3
    elif 10000 <= pop < 50000:
        return 4
    else:
        return 5
    


output_df = []
for i in files:
    #print(i)
    total = {}
    raw_df = pd.read_csv('./dataset/'+i).fillna(0)
    country = [col for col in raw_df.columns if 'Country' in col][0]
    state = [col for col in raw_df.columns if 'Province' in col][0]
    check = raw_df[country] == 'US'
    df = raw_df[check]

    for _, row in df.iterrows():
        us_state = row[state].split(',')[0]
        if us_state not in state_code.CODE:
            if len(row[state].split(',')) < 2 or len((row[state].split(',')[1]).split(' ')[1]) < 2:
                continue
            s_code = (row[state].split(',')[1]).split(' ')[1]
            if s_code in state_code.CODE_R:
                us_state = state_code.CODE_R[s_code]
            else:
                print(row[state])
                continue
        #print(row[state],i)
        if us_state in total:
            #print(total[temp])
            total[us_state][0] += row['Confirmed']
            total[us_state][1] += row['Deaths']
            total[us_state][2] += row['Recovered']
        else:
            total[us_state] = [row['Confirmed'], row['Deaths'], row['Recovered']]
            #total[row[state]][3] += row['Active']
    #print(total)
    #usa_df = pd.DataFrame(columns=['code','total_confirmed','total_death','total_recovered','date'])
    usa_df = pd.DataFrame(columns=['code','state','total_confirmed','color_code', 'date'])

    # total two-d array total[0] = [number_confirmed, number_death, number_ recovered]
    #wtf
    for j in total:
        item = total[j][0]
        biggest_con = max(biggest_con, total[j][0])
        a_row = [state_code.CODE[j], j]+[str(int(item))]+[set_color_group(item)]+ [i[:10]]
        leng = len(usa_df)
        usa_df.loc[leng] = a_row
        #row_df = pd.DataFrame([a_row])
        # usa_df = usa_df.append(a_row, ignore_index=True)
    #print(usa_df)
    output_df.append(usa_df.fillna(0))

97
97


In [62]:


# Create figure
fig = go.Figure()

colors = ['#FFE9E5', '#FFAFA1', '#FF745C' , '#FF745C', '#9A1903']

# Add traces, one for each slider step
for i in output_df:
    #print(i['total_confirmed'].astype(int))
    fig.add_trace(
        go.Choropleth(
    locations=i['code'], # Spatial coordinates
    z = i['color_code'], # Data to be color-coded
    zmin = 0,
    zmax = 6,
    colorscale = 'reds',
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorbar_title = "People",
    hovertext = i['state'] + '<br>' +'Confirmed: '+ i['total_confirmed'],
    colorbar = dict(
        tickvals=[1, 2, 3, 4, 5, 6],
        ticktext=["1000", "5000", "10000", "50000", "100000"]+[str(biggest_con)])
    ))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label= output_df[i]['date'][0][:5])
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Date: "},
    pad={"t": 1},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title_text = 'COVID-19 USA Data Visualization',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [36]:
output_df[-5]

,code,total_confirmed,color_code,date
0,AL,3,5317,04-21-2020
1,AK,1,329,04-21-2020
2,AS,0,0,04-21-2020
3,AZ,3,5256,04-21-2020
4,AR,2,1990,04-21-2020
5,CA,4,35465,04-21-2020
6,CO,4,10473,04-21-2020
7,CT,4,20360,04-21-2020
8,DE,2,2931,04-21-2020
9,DC,0,0,04-21-2020


In [ ]:
def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale
    
bvals = [0, 1000, 5000, 10000, 50000, 1000000]

colors = ['#FFE9E5', '#FFAFA1', '#FF745C' , '#FF745C', '#9A1903']

dcolorsc = discrete_colorscale(bvals, colors)

bvals = np.array(bvals)

tickvals = [np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] #position with respect to bvals where ticktext is displayed

ticktext = [f'<{bvals[1]}'] + [f'{bvals[k]}-{bvals[k+1]}' for k in range(1, len(bvals)-2)]+[f'>{bvals[-2]}']